In [1]:
# Importation des librairies
import os
import random
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from torch.utils.data import DataLoader, Dataset
from IPython.display import Audio, display

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Chemin vers le dossier contenant les fichiers audio et la transcription
DATA_DIR = "/content/drive/MyDrive/datasets/data"
AUDIO_FILES = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.wav')]
TRANSCRIPTION_FILE = os.path.join(DATA_DIR, "transcription.txt")

In [4]:
# Chargement du tokenizer et du modèle pré-entraîné
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you ar

In [5]:
import torchaudio

def resample_audio(file_path, target_sample_rate=16000):
    audio, sample_rate = torchaudio.load(file_path)
    if sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        audio = resampler(audio)
    return audio.squeeze()  #S'assurer que l'audio est mono pour l'utiliser

# Fonction pour lire et rééchantillonner les fichiers audio
def read_and_resample_audio(file_path):
    audio = resample_audio(file_path)
    return audio

In [6]:
# Fonction pour lire les fichiers audio
def read_audio(file_path):
    audio, sample_rate = sf.read(file_path)
    # Convertir en mono si l'audio est stéréo
    if audio.ndim > 1:
        audio = audio.mean(axis=1)
    return audio

# Fonction pour charger les transcriptions
def load_transcriptions(transcription_file):
    with open(transcription_file, "r", encoding="utf-8") as f:
        transcriptions = f.read().splitlines()
    return {line.split()[0]: " ".join(line.split()[1:]) for line in transcriptions}

In [7]:
# Prétraitement des données audio
'''
def preprocess_audio(audio_files):
    audio_data = []
    for file_path in audio_files:
        data = read_and_resample_audio(file_path)
        audio_data.append(data)
    return audio_data  # Retourne une liste de tableaux NumPy
'''
# Prétraitement des données audio
def preprocess_audio(audio_files):
    audio_data = []
    for file_path in audio_files:
        # Lire et rééchantillonner l'audio
        data = read_and_resample_audio(file_path)
        # S'assurer que l'audio est mono en supprimant la première dimension si elle est de taille 2
        if data.shape[0] == 2:
            data = data.mean(axis=0)  # Convertir en mono si nécessaire
        # Ajouter une dimension de canal à la fin si elle n'existe pas
        if data.ndim == 1:
            data = data.unsqueeze(1)
        audio_data.append(data)
    return audio_data  # Retourne une liste de tenseurs avec la forme attendue

# Prétraitement des transcriptions
def preprocess_transcriptions(transcriptions):
    return [processor.tokenizer.encode(transcription, add_special_tokens=True) for transcription in transcriptions.values()]

In [8]:
# Charger les transcriptions
transcriptions = load_transcriptions(TRANSCRIPTION_FILE)

# Prétraiter les données audio et les transcriptions
input_values = preprocess_audio(AUDIO_FILES)
labels = preprocess_transcriptions(transcriptions)

In [9]:
# Verification des dimensions des audios apres Prétraitement
for i, v in enumerate(input_values):
    print(f"Forme des données audio {i}: {v.shape}")

Forme des données audio 0: torch.Size([10550, 1])
Forme des données audio 1: torch.Size([11031, 1])
Forme des données audio 2: torch.Size([9386, 1])
Forme des données audio 3: torch.Size([11031, 1])
Forme des données audio 4: torch.Size([12515, 1])
Forme des données audio 5: torch.Size([11874, 1])
Forme des données audio 6: torch.Size([11793, 1])
Forme des données audio 7: torch.Size([9627, 1])
Forme des données audio 8: torch.Size([9948, 1])
Forme des données audio 9: torch.Size([10269, 1])


In [10]:
'''
# Recalcul des longueurs des séquences d'entrée avant le padding
input_lengths = [len(input) for input in input_values]

# Recalcul des longueurs des cibles avant le padding
target_lengths = [len(target) for target in labels]
'''

"\n# Recalcul des longueurs des séquences d'entrée avant le padding\ninput_lengths = [len(input) for input in input_values]\n\n# Recalcul des longueurs des cibles avant le padding\ntarget_lengths = [len(target) for target in labels]\n"

In [11]:
# Convertir input_values et labels en tenseurs PyTorch
input_values = [torch.tensor(v, dtype=torch.float32) for v in input_values]
#input_values = [torch.tensor(dtype=torch.float32, v) for v in input_values]
labels = [torch.tensor(l, dtype=torch.int32) for l in labels]

<ipython-input-11-47278c278ff9>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = [torch.tensor(v, dtype=torch.float32) for v in input_values]


In [12]:
# Calculez les longueurs des séquences d'entrée et sorties avant le padding
input_lengths = [v.size(0) for v in input_values]
target_lengths = [len(target) for target in labels]

In [13]:
# Vérification des longueurs des séquences audio avant le padding
print("Longueurs des séquences audio avant le padding:")
input_lengths = []
for i, v in enumerate(input_values):
    print(f"Audio {i}: Longueur {v.size(0)}")
    input_lengths.append(v.size(0))

Longueurs des séquences audio avant le padding:
Audio 0: Longueur 10550
Audio 1: Longueur 11031
Audio 2: Longueur 9386
Audio 3: Longueur 11031
Audio 4: Longueur 12515
Audio 5: Longueur 11874
Audio 6: Longueur 11793
Audio 7: Longueur 9627
Audio 8: Longueur 9948
Audio 9: Longueur 10269


In [14]:
# Vérification des longueurs des étiquettes avant le padding
print("Longueurs des étiquettes avant le padding:")
target_lengths = []
for i, l in enumerate(labels):
    print(f"Étiquette {i}: Longueur {l.size(0)}")
    target_lengths.append(l.size(0))

Longueurs des étiquettes avant le padding:
Étiquette 0: Longueur 3
Étiquette 1: Longueur 4
Étiquette 2: Longueur 4
Étiquette 3: Longueur 3
Étiquette 4: Longueur 4
Étiquette 5: Longueur 6
Étiquette 6: Longueur 7
Étiquette 7: Longueur 4
Étiquette 8: Longueur 5
Étiquette 9: Longueur 4
Étiquette 10: Longueur 5


In [15]:
# S'assurer que chaque tenseur audio a deux dimensions
input_values = [v.unsqueeze(1) if v.dim() == 1 else v for v in input_values]

# Vérifier les dimensions des tenseurs
for i, v in enumerate(input_values):
    print(f"Audio {i}: Dimensions {v.size()}")

Audio 0: Dimensions torch.Size([10550, 1])
Audio 1: Dimensions torch.Size([11031, 1])
Audio 2: Dimensions torch.Size([9386, 1])
Audio 3: Dimensions torch.Size([11031, 1])
Audio 4: Dimensions torch.Size([12515, 1])
Audio 5: Dimensions torch.Size([11874, 1])
Audio 6: Dimensions torch.Size([11793, 1])
Audio 7: Dimensions torch.Size([9627, 1])
Audio 8: Dimensions torch.Size([9948, 1])
Audio 9: Dimensions torch.Size([10269, 1])


In [16]:
# Padding des séquences audio pour qu'elles aient toutes la même longueur
input_values_padded = torch.nn.utils.rnn.pad_sequence([v.squeeze(0) for v in input_values], batch_first=True)

In [17]:
# Application du padding aux étiquettes
labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id)

In [18]:
# Création d'un Dataset personnalisé
class AudioData(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

# Création du DataLoader
train_dataset = AudioData(input_values_padded, labels_padded)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [19]:
# Initialisation du processor qui contient le tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

In [20]:
# Définition de la fonction de perte et de l'optimiseur
loss_fn = torch.nn.CTCLoss(blank=processor.tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

In [21]:
# Boucle d'entraînement
model.train()
for epoch in range(10):  # Nombre d'époques
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        # Suppression de la dimension supplémentaire avant de passer les données au modèle
        inputs = inputs.squeeze(-1)

        outputs = model(inputs).logits

        # Calcul des longueurs des séquences d'entrée pour le lot actuel
        current_batch_size = inputs.size(0)
        input_lengths_tensor = torch.tensor(input_lengths[batch_idx * current_batch_size : (batch_idx + 1) * current_batch_size], dtype=torch.int64)

        # Calcul des longueurs des cibles pour le lot actuel
        target_lengths_tensor = torch.tensor(target_lengths[batch_idx * current_batch_size : (batch_idx + 1) * current_batch_size], dtype=torch.int64)

        # Aplatir les cibles en un vecteur 1D
        targets_flat = torch.cat([t for t in targets]).to(outputs.device)

        # Calcul de la perte en utilisant la fonction de perte CTC
        loss = loss_fn(outputs.log_softmax(2), targets_flat, input_lengths_tensor, target_lengths_tensor)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} Batch {batch_idx} Loss: {loss.item()}")


RuntimeError: input_lengths must be of size batch_size

In [ ]:
# Fonction pour jouer un fichier audio au hasard avec sa transcription
def play_random_audio(transcriptions, audio_files):
    random_index = random.randint(0, len(audio_files) - 1)
    random_audio_path = audio_files[random_index]
    random_audio_id = os.path.basename(random_audio_path).split('.')[0]
    audio_data = read_audio(random_audio_path)
    print(f"Transcription pour {random_audio_id}: {transcriptions[random_audio_id]}")
    display(Audio(data=audio_data, rate=16000))

# Jouer un fichier audio au hasard
play_random_audio(transcriptions, AUDIO_FILES)